In [68]:
import faiss
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import Settings
from llama_index.core import get_response_synthesizer
import logging
import sys, os

from dotenv import load_dotenv
load_dotenv(".env")

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [37]:
# dimensions of text-ada-embedding-002
d = 1536
faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# vector_store = FaissVectorStore(faiss_index=faiss_index).from_persist_dir("./storage")
# storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir="./storage")

In [5]:
llm = AzureOpenAI(
    model=os.getenv("AZURE_OPENAI_MODEL"),
    deployment_name=os.getenv("AZURE_OPENAI_NAME"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_VERSION"),
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model=os.getenv("AZURE_EMBEDDING_MODEL"),
    deployment_name=os.getenv("AZURE_EMBEDDING_NAME"),
    api_key=os.getenv("AZURE_EMBEDDING_KEY"),
    azure_endpoint=os.getenv("AZURE_EMBEDDING_ENDPOINT"),
    api_version=os.getenv("AZURE_EMBEDDING_VERSION"),
)

Settings.llm = llm
Settings.embed_model = embed_model

# llamaIndex
- Example: Load resume and read from it
- Document load -> Vector Store -> Query
- https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/

## Document Load
- https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/

In [6]:
reader = SimpleDirectoryReader(input_files=["./data/my_resume.pdf"])
documents = reader.load_data()
documents

[Document(id_='897d932f-3b5a-4a1b-9d8f-e157043f84bf', embedding=None, metadata={'page_label': '1', 'file_name': 'my_resume.pdf', 'file_path': 'data/my_resume.pdf', 'file_type': 'application/pdf', 'file_size': 250772, 'creation_date': '2025-04-27', 'last_modified_date': '2025-04-27'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text="GAN SHAO HONG \n(+65) 90855781 | shaohong.g97@gmail.com | www.linkedin.com/in/g-shaohong/ | https://github.com/shaohong-g  \n \nPROFESSIONAL SUMMARY \n• Machine Learning Engineer with 3+ years of experience in developing highly scalable AI models, implementing ML \npipelines, and deploying models in cloud environ

In [8]:
vars(documents[0])

{'id_': '897d932f-3b5a-4a1b-9d8f-e157043f84bf',
 'embedding': None,
 'metadata': {'page_label': '1',
  'file_name': 'my_resume.pdf',
  'file_path': 'data/my_resume.pdf',
  'file_type': 'application/pdf',
  'file_size': 250772,
  'creation_date': '2025-04-27',
  'last_modified_date': '2025-04-27'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {},
 'metadata_template': '{key}: {value}',
 'metadata_separator': '\n',
 'text_resource': MediaResource(embeddings=None, data=None, text="GAN SHAO HONG \n(+65) 90855781 | shaohong.g97@gmail.com | www.linkedin.com/in/g-shaohong/ | https://github.com/shaohong-g  \n \nPROFESSIONAL SUMMARY \n• Machine Learning Engineer with 3+ years of experience in developing highly scalable AI models, implemen

## Insert parsed documents to vectorstore

In [38]:
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
index.storage_context.persist()

INFO:httpx:HTTP Request: POST https://black-exposure.openai.azure.com/openai/deployments/black-ada-002/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://black-exposure.openai.azure.com/openai/deployments/black-ada-002/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://black-exposure.openai.azure.com/openai/deployments/black-ada-002/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://black-exposure.openai.azure.com/openai/deployments/black-ada-002/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://black-exposure.openai.azure.com/openai/deployments/black-ada-002/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://black-exposure.openai.azure.com/openai/deployments/black-ada-002/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://black-exposure.openai.azure.com/openai/deployments/black-ada-002/embeddings?api-version=2023-05-15

In [39]:
index = load_index_from_storage(storage_context=storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


In [ ]:
query = "How many jobs have the applicants done? Give the company name, role as well as time."
query = "How many jobs have the applicants done from 2022-2023?"

response_synthesizer = get_response_synthesizer(response_mode="compact") # refine , context_only, accumulate, compact (default)
query_engine = index.as_query_engine(response_synthesizer=response_synthesizer)
response = query_engine.query(query)

INFO:httpx:HTTP Request: POST https://black-exposure.openai.azure.com/openai/deployments/black-ada-002/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://black-exposure.openai.azure.com/openai/deployments/black-ada-002/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://black-exposure.openai.azure.com/openai/deployments/black-ada-002/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://black-exposure.openai.azure.com/openai/deployments/black-ada-002/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://black-exposure.openai.azure.com/openai/deployments/black-ada-002/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://black-exposure.openai.azure.com/openai/deployments/black-ada-002/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://black-exposure.openai.azure.com/openai/deployments/black-ada-002/embeddings?api-version=2023-05-15

In [81]:

print(response.get_formatted_sources())
print("query was:", query)
print("answer was:", response)

> Source (Doc id: a6455d01-5069-484e-ab17-94bf8c26202e): Neuron Mobility  
Data Engineering Intern May 2022 – Aug 2022 
• Pipeline migration from existing...

> Source (Doc id: 153458dc-dc80-4b58-b479-11b22aea1ee8): GAN SHAO HONG 
(+65) 90855781 | shaohong.g97@gmail.com | www.linkedin.com/in/g-shaohong/ | https:...
query was: How many jobs have the applicants done from 2022-2023?
answer was: The applicant has done four jobs from 2022 to 2023:

1. **Neuron Mobility** (Data Engineering Intern) - May 2022 to August 2022  
2. **Infineon Technologies** (Machine Learning and Test Verification Intern) - August 2022 to January 2023  
3. **Asurion** (Intern – Technology, Data Science, AI & Machine Learning) - January 2023 to May 2023  
4. **UBS** (Software Engineer, full-stack) - August 2023 to January 2025 (part of this role falls in 2023).


In [ ]:
import json
nodes = [json.loads(x.to_json()) for x in response.source_nodes]
node1 = nodes[0]
node1.keys()

dict_keys(['node', 'score', 'class_name'])

In [63]:
print(node1["score"], nodes[1]["score"])
node1["node"]

0.48575446009635925 0.4998507499694824


{'id_': 'a6455d01-5069-484e-ab17-94bf8c26202e',
 'embedding': None,
 'metadata': {'page_label': '2',
  'file_name': 'my_resume.pdf',
  'file_path': 'data/my_resume.pdf',
  'file_type': 'application/pdf',
  'file_size': 250772,
  'creation_date': '2025-04-27',
  'last_modified_date': '2025-04-27'},
 'excluded_embed_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'excluded_llm_metadata_keys': ['file_name',
  'file_type',
  'file_size',
  'creation_date',
  'last_modified_date',
  'last_accessed_date'],
 'relationships': {'1': {'node_id': 'abb5dcdd-6203-42a7-99fe-16177741713b',
   'node_type': '4',
   'metadata': {'page_label': '2',
    'file_name': 'my_resume.pdf',
    'file_path': 'data/my_resume.pdf',
    'file_type': 'application/pdf',
    'file_size': 250772,
    'creation_date': '2025-04-27',
    'last_modified_date': '2025-04-27'},
   'hash': '05b49a6b9291d425546cdaa3a50e53191769ff57a18d8517f3f8ce58934